In [1]:
# imports and settings

import os
import time
import pickle
import warnings
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import numpy as np
from numpy import linalg as LA
from numpy import histogram2d

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt, welch
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import utils as ut
%load_ext autoreload
%autoreload 2

# do not show warnings
warnings.filterwarnings("ignore")

print("Imports complete.")

names, fss, all_data, recommended_slices = ut.load_ds_samples()

Imports complete.
Settings: height=800, width=1400, font_size=16
Imports complete.
Data dicts: 
names=['dpv1', 'dpv2', 'croatia_ship', 'bg_noise']
fss keys=['dpv1', 'dpv2', 'croatia_ship', 'bg_noise']
all_data keys=['dpv1', 'dpv2', 'croatia_ship', 'bg_noise']
slices keys=['dpv1', 'dpv2', 'croatia_ship', 'bg_noise']


In [32]:
duration = 10  # seconds
fs = 8000  # Hz
t = np.linspace(0, duration, int(fs*duration), endpoint=False)
freq1 = 440  # A4
freq2 = 880  # A5
signal1 = 0.5 * np.sin(2 * np.pi * freq1 * t)
signal2 = 0.5 * np.sin(2 * np.pi * freq2 * t)
composite_signal = signal1 + signal2 + 0.1 * np.random.normal(size=t.shape)

#spectrogram
F, T, Sxx, _ = ut.calc_spectrogram(composite_signal, fs, nperseg=1024, percent_overlap=0.5, window='hamming', remove_dc=20, crop_freq=1000)
Sxx = 10 * np.log10(Sxx + 1e-10)
fig = go.Figure(data=go.Heatmap(z=Sxx.T, x=T, y=F, colorscale='Viridis'))
fig.update_layout(title='Spectrogram of Composite Signal', xaxis_title='Time (s)', yaxis_title='Frequency (Hz)', height=600, width=800)
fig.show()

In [19]:
mi = ut.mutual_info_matrix(Sxx)

In [20]:
np.fill_diagonal(mi, 0)
fig = go.Figure(data=go.Heatmap(z=mi, x=F, y=F, colorscale='Viridis'))
fig.update_layout(title='Mutual Information Matrix - Composite Signal Spectrogram', xaxis_title='Frequency Bin', yaxis_title='Frequency Bin', height=600, width=600)
fig.show()

In [21]:
# compute correlation matrix between frequency bins
correlation_matrix = np.corrcoef(Sxx)
np.fill_diagonal(correlation_matrix, 0)
fig = go.Figure(data=go.Heatmap(z=correlation_matrix, x=F, y=F, colorscale='Viridis'))
fig.update_layout(title='Correlation Matrix - Composite Signal Spectrogram', xaxis_title='Frequency Bin', yaxis_title='Frequency Bin', height=600, width=600)
fig.show()

In [22]:
inverse_mi = 1 / (mi + 1e-10)  # add small value to avoid division by zero
np.fill_diagonal(inverse_mi, 0)
fig = go.Figure(data=go.Heatmap(z=inverse_mi, x=F, y=F, colorscale='Viridis'))
fig.update_layout(title='Inverse Mutual Information Matrix - Composite Signal Spectrogram', xaxis_title='Frequency Bin', yaxis_title='Frequency Bin', height=600, width=600)
fig.show()

In [49]:
# calculate frequency predictivity scores: how well S(t, f_i) predicts S(t, f_j)
num_freq_bins = len(F)
predictivity_scores = np.zeros((num_freq_bins, num_freq_bins))
for i in range(num_freq_bins):
    for j in range(num_freq_bins):
        ti = Sxx[:, i][:-1]
        tj = Sxx[:, j][1:]
        # p = np.corrcoef(ti, tj)[0, 1] ** 2  # 
        p = ut.mutual_information(ti, tj)  # 
        predictivity_scores[i, j] = p

fig = go.Figure(data=go.Heatmap(z=predictivity_scores, x=F, y=F, colorscale='Viridis'))
fig.update_layout(title='Frequency Predictivity Scores - Composite Signal Spectrogram', xaxis_title='Frequency Bin', yaxis_title='Frequency Bin', height=600, width=600)
fig.show()

In [27]:
entropies = []

for i in range(Sxx.shape[1]):
    freq_bin_signal = Sxx[:, i]
    entropies.append(ut.entropy(freq_bin_signal))

fig = go.Figure(data=go.Scatter(x=F, y=entropies, mode='lines+markers'))
fig.update_layout(title='Entropy of Frequency Bins - Composite Signal Spectrogram', xaxis_title='Frequency (Hz)', yaxis_title='Entropy', height=400, width=600)
fig.show()

In [33]:
#spectrogram
F, T, Sxx, _ = ut.calc_spectrogram(composite_signal, fs, nperseg=1024, percent_overlap=0.5, window='hamming', remove_dc=20, crop_freq=1000)
Sxx = 10 * np.log10(Sxx + 1e-10)
fig = go.Figure(data=go.Heatmap(z=Sxx.T, x=T, y=np.arange(len(F)), colorscale='Viridis'))
fig.update_layout(title='Spectrogram of Composite Signal', xaxis_title='Time (s)', yaxis_title='Frequency (Hz)', height=600, width=800)
fig.show()

In [40]:
all_f_hist, all_bins = np.histogram(Sxx, bins=50)
all_f_variance = np.var(Sxx)
all_f_entropy = ut.entropy(Sxx.flatten())
f1_hist, f1_bins = np.histogram(Sxx[:, 56], bins=50)
f1_variance = np.var(Sxx[:, 56])
f1_entropy = ut.entropy(Sxx[:, 56])
f2_hist, f2_bins = np.histogram(Sxx[:, 26], bins=50)
f2_variance = np.var(Sxx[:, 26])
f2_entropy = ut.entropy(Sxx[:, 26])

subplot_titles = [
    f'Variance={all_f_variance:.2f}, Entropy={all_f_entropy:.2f}',
    f'Variance={f1_variance:.2f}, Entropy={f1_entropy:.2f}', 
    f'Variance={f2_variance:.2f}, Entropy={f2_entropy:.2f}'
]


fig = make_subplots(cols=3, rows=1, subplot_titles=subplot_titles)
fig.add_trace(go.Bar(x=all_bins[:-1], y=all_f_hist, name='All Frequencies', marker_color='blue', opacity=0.6), row=1, col=1)
fig.add_trace(go.Bar(x=f1_bins[:-1], y=f1_hist, name='Frequency Bin 56 (signal)', marker_color='red', opacity=0.6), row=1, col=2)
fig.add_trace(go.Bar(x=f2_bins[:-1], y=f2_hist, name='Frequency Bin 26 (noise)', marker_color='green', opacity=0.6), row=1, col=3)
fig.update_layout(title='Histogram of Spectrogram Values', height=400, width=1200)
fig.show()

In [46]:
fig = make_subplots(rows=1, cols=5, shared_yaxes=True, column_widths=[0.6, 0.1, 0.1, 0.1, 0.1], horizontal_spacing=0.02, subplot_titles=['Spectrogram', 'Mean Power', 'Entropy', 'Std Dev', 'Variance'])

pxx = np.mean(Sxx, axis=0)
entropies = [ut.entropy(Sxx[:, i]) for i in range(Sxx.shape[1])]
std = np.std(Sxx, axis=0)
variance = np.var(Sxx, axis=0)

fig.add_trace(go.Heatmap(z=Sxx.T, x=T, y=np.arange(len(F)), colorscale='Viridis'), row=1, col=1)
fig.add_trace(go.Scatter(x=pxx, mode='lines', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=entropies, mode='lines', line=dict(color='orange')), row=1, col=3)
fig.add_trace(go.Scatter(x=std, mode='lines', line=dict(color='green')), row=1, col=4)
fig.add_trace(go.Scatter(x=variance, mode='lines', line=dict(color='red')), row=1, col=5)
fig.update_layout(title='Spectrogram of Composite Signal', xaxis_title='Time (s)', yaxis_title='Frequency (Hz)', height=600, width=800)
fig.show()

In [50]:
# calculate frequency predictivity scores: how well S(t, f_i) predicts S(t, f_j)

def predictivity(S):
    num_freq_bins = S.shape[1]
    predictivity_scores = np.zeros((num_freq_bins, num_freq_bins))
    for i in range(num_freq_bins):
        for j in range(num_freq_bins):
            ti = S[:, i][:-1]
            tj = S[:, j][1:]
            p = np.sqrt(((ti - tj) ** 2).mean())  # mean squared error
            predictivity_scores[i, j] = p
    return predictivity_scores


predictivity_scores = predictivity(Sxx)

fig = go.Figure(data=go.Heatmap(z=predictivity_scores, x=F, y=F, colorscale='Viridis'))
fig.update_layout(title='Frequency Predictivity Scores - Composite Signal Spectrogram', xaxis_title='Frequency Bin', yaxis_title='Frequency Bin', height=600, width=600)
fig.show()